## 크롤링 준비 & 이미지 스크래핑 & 자연어 처리

- 이미지 스크래핑

In [69]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import HTTPError
from urllib.error import URLError
import pandas as pd
from urllib.parse import quote_plus

In [70]:
base_url = 'https://search.naver.com/search.naver?where=image&sm=tab_jum&query='
keyword = input('검색어 입력 : ')
imgCnt = int(input('스크랩할 이미지 개수 : '))

try :
    # quote_plus : 문자를 한국어로 인식하게 해주는 함수
    url = base_url + quote_plus(keyword)
    html = urlopen(url)
except HTTPError as he :
    print('http error')
except URLError as ue :
    print('url error')
else :
    soup = BeautifulSoup(html.read(), 'html.parser')

검색어 입력 : 펭수
스크랩할 이미지 개수 : 50


In [71]:
img = soup.find_all('img', '_img')
# img

In [72]:
cnt = 1
for i in img :
    imgUrl = i['data-source']
    
    with urlopen(imgUrl) as file :
        # wb : write_binary. 사진을 읽어올때 주로 사용
        with open('./images/' + str(cnt) + '.jpg', 'wb') as imgFile :
            img = file.read()
            imgFile.write(img)
    
    cnt += 1
    if cnt > imgCnt :
        break

print('image download success')

image download success


- 검색 키워드를 이용하여 원하는 제목과 링크만 가져오는 실습
- csv 파일로 판다스의 DataFrame을 활용해서 저장

In [73]:
base_url = 'https://search.naver.com/search.naver?where=post&sm=tab_jum&query='
keyword = input('검색어 입력 : ')

try :
    # quote_plus : 문자를 한국어로 인식하게 해주는 함수
    url = base_url + quote_plus(keyword)
    html = urlopen(url)
except HTTPError as he :
    print('http error')
except URLError as ue :
    print('url error')
else :
    soup = BeautifulSoup(html.read(), 'html.parser')

검색어 입력 : 손흥민


In [76]:
titles = soup.find_all('a', {'class' : 'sh_blog_title'})
# titles

title_list = []
link_list = []

for title in titles :
    t = title['title']
    # t = title.attrs['title']
    title_list.append(t)
    
    l = title['href']
    link_list.append(l)
    

df = pd.DataFrame({
    'title' : title_list,
    'link' : link_list
})

df.head()

,title,link
0,[영어과외] 토트넘 손흥민 요리스 싸움 | 화풀이하다 영어로 take it out on,https://blog.naver.com/tobemaven?Redirect=Log&...
1,[토트넘 맨유] 손흥민 멀티골 작렬! 2골 1어시 !(feat.유럽 리그 통산 100골),https://blog.naver.com/vpahs?Redirect=Log&logN...
2,"손흥민 맨유전 2골 1어시, 득점왕을 노려! (일본반응)",https://blog.naver.com/slayer_tr?Redirect=Log&...
3,유일하게 신나는 뉴스 손흥민 네골 득점!,https://lkl7383.blog.me/222096282723
4,손흥민 잉글랜드 프리미어리그 EPL 최다 4골 폭풍 토트넘 케인 어시스트,https://blog.naver.com/mobile8889?Redirect=Log...


In [77]:
# index = False
# 불필요한 인덱싱을 넣지 않고 저장
df.to_csv('./data/title.csv', mode = 'w', encoding = 'utf-8', index = False)

df = pd.read_csv('./data/title.csv', encoding = 'utf-8')
df.head()

,title,link
0,[영어과외] 토트넘 손흥민 요리스 싸움 | 화풀이하다 영어로 take it out on,https://blog.naver.com/tobemaven?Redirect=Log&...
1,[토트넘 맨유] 손흥민 멀티골 작렬! 2골 1어시 !(feat.유럽 리그 통산 100골),https://blog.naver.com/vpahs?Redirect=Log&logN...
2,"손흥민 맨유전 2골 1어시, 득점왕을 노려! (일본반응)",https://blog.naver.com/slayer_tr?Redirect=Log&...
3,유일하게 신나는 뉴스 손흥민 네골 득점!,https://lkl7383.blog.me/222096282723
4,손흥민 잉글랜드 프리미어리그 EPL 최다 4골 폭풍 토트넘 케인 어시스트,https://blog.naver.com/mobile8889?Redirect=Log...


## 크롤링

In [78]:
from time import sleep
from random import randint
import pandas as pd
from IPython.core.display import clear_output
from time import time
from bs4 import BeautifulSoup
from requests import get

In [79]:
# loop 사이에 잘 진행되고 있는지를 보기 위한 indicator sentence
# 연습 단계
start_time = time()
# start_time
requests = 0

# 계속적으로 바뀌는 requests를 넣는데는 시간이 필요하다는 것을 확인하기 위한 단계
for _ in range(5) :
    requests += 1
    sleep(randint(1, 3))
    current_time = time()
    elapsed_time = current_time - start_time
    print('request : {}, frequency : {}'.format(requests, requests / elapsed_time))
    # clear_output(wait = True)

request : 1, frequency : 0.4963505964591727
request : 2, frequency : 0.49812010106806004
request : 3, frequency : 0.5980182421790734
request : 4, frequency : 0.663277984578086
request : 5, frequency : 0.6217918296960604


In [80]:
base_url = 'https://search.naver.com/search.naver?where=post&sm=tab_jum&query='
keyword = input('검색어 입력 : ')

try :
    # quote_plus : 문자를 한국어로 인식하게 해주는 함수
    url = base_url + quote_plus(keyword)
    html = urlopen(url)
except HTTPError as he :
    print('http error')
except URLError as ue :
    print('url error')
else :
    soup = BeautifulSoup(html.read(), 'html.parser')

검색어 입력 : 그래픽카드


In [81]:
titles = soup.find_all('a', {'class' : 'sh_blog_title'})
# titles

In [82]:
pages = [str(i) for i in range(1, 11)]
pages

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [83]:
# loop 사이에 잘 진행되고 있는지를 보기 위한 indicator sentence
start_time = time()
requests = 0

price_list = []
link_list = []
title_list = []

for page in pages :
    requests += 1
    
    url = get('https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery=%EA%B7%B8%EB%9E%98%ED%94%BD%EC%B9%B4%EB%93%9C&pagingIndex=' + page + '&pagingSize=40&productSet=total&query=%EA%B7%B8%EB%9E%98%ED%94%BD%EC%B9%B4%EB%93%9C&sort=rel&timestamp=&viewType=list')
    
    # request 에 delaytime 을 준다.
    sleep(randint(1, 3))
    current_time = time()
    elapsed_time = current_time - start_time
    
    soup = BeautifulSoup(url.text, 'html.parser')
    lis = soup.find_all('li', 'basicList_item__2XT81')
    
    for li in lis :
        # title, link, price
        title_list.append(li.find('a', 'basicList_link__1MaTN').text)
        price_list.append(li.find('span', 'price_num__2WUXn').text)
        link_list.append(li.find('a', 'basicList_link__1MaTN').attrs['href'])
    
# print(len(title_list))
# print(len(price_list))
# print(len(link_list))

In [64]:
# 다른 방법

price_list = []
title_list = []
link_list  = []

# loop 사이에 잘 진행되고 있는지를 보기위한  indicator sentence
start_time = time()
requests = 0
for page in pages :
    requests += 1
    
    url = get('https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery=%EA%B7%B8%EB%9E%98%ED%94%BD%EC%B9%B4%EB%93%9C&pagingIndex=' + page + '&pagingSize=40&productSet=total&query=%EA%B7%B8%EB%9E%98%ED%94%BD%EC%B9%B4%EB%93%9C&sort=rel&timestamp=&viewType=list')
    sleep(randint(1,3))
    current_time = time()
    elapsed_time = current_time - start_time
    
    soup = BeautifulSoup(url.text, 'html.parser')
    # title, link, price
    # 가격 가져오기
    prices = soup.select('.price_num__2WUXn')
    prices_text = [price.text for price in prices]
    price_list.extend(prices_text)
    
    # 타이틀, 링크 가져오기
    titles = soup.select('.basicList_link__1MaTN')
    titles_text = [title.text for title in titles]
    title_list.extend(titles_text)

    link_text = [title['href'] for title in titles]
    link_list.extend(link_text)

# print(len(title_list))
# print(len(price_list))
# print(len(link_list))

In [84]:
python_page_df = pd.DataFrame({
    'title' : title_list,
    'price' : price_list,
    'link' : link_list
})

python_page_df.head()

,title,price,link
0,조텍 GAMING 지포스 GTX 1660 SUPER AMP D6 6GB 백플레이트,"295,890원",https://adcr.naver.com/adcr?x=cIBtNz0L8l+oM5UR...
1,조텍 RTX 2060 SUPER OC D6 8GB White Edition,"525,190원",https://adcr.naver.com/adcr?x=PZVpf0geW4ylwyDM...
2,이엠텍 지포스 GTX 1660 SUPER STORM X Dual OC D6 6GB,"289,470원",https://adcr.naver.com/adcr?x=igreobLuhQv5lZiZ...
3,MSI 지포스 GTX1050 Ti OC D5 4GB 윈드스톰,"166,970원",https://cr.shopping.naver.com/adcr.nhn?x=BWxUL...
4,이엠텍 지포스 GTX 1660 SUPER STORM X Dual OC D6 6GB,"289,470원",https://cr.shopping.naver.com/adcr.nhn?x=RiVAy...


In [85]:
python_page_df.to_csv('./data/GraphicCards.csv', mode = 'w', encoding = 'utf-8', index = False)

df = pd.read_csv('./data/GraphicCards.csv', encoding = 'utf-8')
df.head()

,title,price,link
0,조텍 GAMING 지포스 GTX 1660 SUPER AMP D6 6GB 백플레이트,"295,890원",https://adcr.naver.com/adcr?x=cIBtNz0L8l+oM5UR...
1,조텍 RTX 2060 SUPER OC D6 8GB White Edition,"525,190원",https://adcr.naver.com/adcr?x=PZVpf0geW4ylwyDM...
2,이엠텍 지포스 GTX 1660 SUPER STORM X Dual OC D6 6GB,"289,470원",https://adcr.naver.com/adcr?x=igreobLuhQv5lZiZ...
3,MSI 지포스 GTX1050 Ti OC D5 4GB 윈드스톰,"166,970원",https://cr.shopping.naver.com/adcr.nhn?x=BWxUL...
4,이엠텍 지포스 GTX 1660 SUPER STORM X Dual OC D6 6GB,"289,470원",https://cr.shopping.naver.com/adcr.nhn?x=RiVAy...
